## Purpose
This notebook is to get the bounding boxes from the ground truth labels and visualize the bouding boxes. We can cross check the bounding box generated from the original images and masks. We can also identify whether there is any all black masks in our dataset (so 0 bounding box is generated).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from glob import glob
from tqdm.notebook import tqdm

## Move data to local disk

**IMPORTANT:** Replace your own dataset paths

In [ ]:
zip_train_source_path = "/content/drive/MyDrive/FYP/Datasets/zipped/train.zip"
zip_val_source_path = "/content/drive/MyDrive/FYP/Datasets/zipped/validation.zip"

local_data_dir = "/content/data"

!mkdir -p "$local_data_dir"

print(f"Copying {zip_train_source_path} to {local_data_dir}")
!cp "$zip_train_source_path" "$local_data_dir/"

print(f"Copying {zip_val_source_path} to {local_data_dir}")
!cp "$zip_val_source_path" "$local_data_dir/"

print("Copying complete.")

In [ ]:
local_zip_train_path = f"{local_data_dir}/train.zip"
local_zip_val_path = f"{local_data_dir}/validation.zip"

unzip_destination_path = local_data_dir

print(f"Unzipping {local_zip_train_path} to {unzip_destination_path}")
!unzip -q "$local_zip_train_path" -d "$unzip_destination_path"

print(f"Unzipping {local_zip_val_path} to {unzip_destination_path}")
!unzip -q "$local_zip_val_path" -d "$unzip_destination_path"

print("Unzipping complete.")

In [ ]:
local_train_image_path = os.path.join(local_data_dir, "train", "images")
local_train_mask_path = os.path.join(local_data_dir, "train", "masks")

# Check if directories exist
if os.path.exists(local_train_image_path):
  num_images = len(os.listdir(local_train_image_path))
  print(f"Number of images in {local_train_image_path}: {num_images}")
else:
  print(f"Directory {local_train_image_path} does not exist.")

if os.path.exists(local_train_mask_path):
  num_masks = len(os.listdir(local_train_mask_path))
  print(f"Number of masks in {local_train_mask_path}: {num_masks}")
else:
  print(f"Directory {local_train_mask_path} does not exist.")

In [ ]:
local_val_image_path = os.path.join(local_data_dir, "validation", "images")
local_val_mask_path = os.path.join(local_data_dir, "validation", "masks")

# Check if directories exist
if os.path.exists(local_val_image_path):
  num_images = len(os.listdir(local_val_image_path))
  print(f"Number of images in {local_val_image_path}: {num_images}")
else:
  print(f"Directory {local_val_image_path} does not exist.")

if os.path.exists(local_val_mask_path):
  num_masks = len(os.listdir(local_val_mask_path))
  print(f"Number of masks in {local_val_mask_path}: {num_masks}")
else:
  print(f"Directory {local_val_mask_path} does not exist.")

## Helper functions

In [ ]:
def mask_to_multiple_bboxes(mask_path, padding_factor=0.1, min_area_threshold=5):
    """
    Converts a binary segmentation mask with multiple disconnected regions
    into a list of bounding boxes (xyxy), one for each region.
    Args:
        mask_path (str): File path to the 2D binary mask image.
        min_area_threshold (int): Minimum pixel area for a component to be considered a valid wound.
    Returns:
        list: A list of bounding boxes, where each box is [x_min, y_min, x_max, y_max] (float32).
    """
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        return [] # Return empty list if mask not found

    H, W = mask.shape
    print(f"Height: {H} and Width: {W}")


    # Ensure the mask is binary (0 or 255)
    _, binary_mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

    # Connected Component Analysis (8-connectivity)
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(binary_mask, 8, cv2.CV_32S)

    bounding_boxes = []

    # Iterate through each component (starting from 1 to skip background)
    for i in range(1, num_labels):
        x = stats[i, cv2.CC_STAT_LEFT]
        y = stats[i, cv2.CC_STAT_TOP]
        w = stats[i, cv2.CC_STAT_WIDTH]
        h = stats[i, cv2.CC_STAT_HEIGHT]
        area = stats[i, cv2.CC_STAT_AREA]

        if area >= min_area_threshold:
            # 1. Determine Padding amount
            # Base padding on the larger dimension for balanced expansion
            padding_pixels = int(max(w, h) * padding_factor)

            # 2. Calculate PADDED coordinates
            x_min_padded = x - padding_pixels
            y_min_padded = y - padding_pixels
            x_max_padded = x + w + padding_pixels
            y_max_padded = y + h + padding_pixels

            # 3. Constrain to Image Boundaries (Crucial step!)
            x_min_final = max(0, x_min_padded)
            y_min_final = max(0, y_min_padded)
            x_max_final = min(W, x_max_padded)
            y_max_final = min(H, y_max_padded)

            # Ensure the box is still valid (min < max)
            if x_min_final < x_max_final and y_min_final < y_max_final:
                # Store as integers for drawing/finetuning
                bbox = [int(x_min_final), int(y_min_final), int(x_max_final), int(y_max_final)]
                bounding_boxes.append(bbox)

    return bounding_boxes

In [ ]:
def visualize_bbox_on_image(image_path, bboxes, save_path=None):
    """Loads an image and draws bounding boxes on it for visualization."""
    # 1. Load the image (OpenCV loads BGR by default, we convert to RGB)
    img = cv2.imread(image_path)
    if img is None:
        print(f"Warning: Image not found at {image_path}")
        return

    # Convert BGR to RGB for Matplotlib
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # 2. Draw Bounding Boxes
    output_img = img_rgb.copy()

    # Define color (e.g., green for visibility)
    color = (0, 255, 0) # RGB Green
    thickness = 3 # Line thickness

    for bbox in bboxes:
        x_min, y_min, x_max, y_max = [int(val) for val in bbox]

        # Draw the rectangle: cv2.rectangle(img, pt1, pt2, color, thickness)
        # pt1 is (x_min, y_min), pt2 is (x_max, y_max)
        cv2.rectangle(output_img, (x_min, y_min), (x_max, y_max), color, thickness)

    # 3. Display or Save
    if save_path:
        # For saving with cv2, convert back to BGR
        cv2.imwrite(save_path, cv2.cvtColor(output_img, cv2.COLOR_RGB2BGR))
        print(f"Saved visualization to: {save_path}")
    else:
        plt.figure(figsize=(10, 8))
        plt.imshow(output_img)
        plt.title(f"Image: {os.path.basename(image_path)} ({len(bboxes)} box(es))")
        plt.axis('off')
        plt.show()

## Loop through all training images

In [ ]:
image_files = glob(os.path.join(local_train_image_path, '*'))
num_files = len(image_files)
print(f"Found {num_files} images to process.")

Found 1019 images to process.


In [ ]:
verification_results = {}

In [ ]:
MASK_EXTENSION = '.png'

for img_path in tqdm(image_files, desc="Verifying Bounding Boxes"):

    full_filename = os.path.basename(img_path)
    base_name_no_ext = os.path.splitext(full_filename)[0]
    mask_filename = base_name_no_ext + MASK_EXTENSION
    mask_path = os.path.join(local_train_mask_path, mask_filename)


    if not os.path.exists(mask_path):
        verification_results[full_filename] = "MASK NOT FOUND"
        continue

    # Get Bounding Boxes
    try:
        bboxes = mask_to_multiple_bboxes(mask_path, min_area_threshold=5)

        # Visualize and Save
        visualize_bbox_on_image(img_path, bboxes)
        verification_results[full_filename] = f"SUCCESS: {len(bboxes)} box(es)"

    except Exception as e:
        verification_results[full_filename] = f"ERROR: {e}"

print("\nBatch Verification Complete.")

In [ ]:
len(verification_results)

In [ ]:
verification_results

In [ ]:
i = 0

for key, value in verification_results.items():
  if value == "SUCCESS: 0 box(es)":
    print(f"Key: {key}, Value: {value}")
    i += 1

## Loop through all validation data

In [ ]:
val_image_files = glob(os.path.join(local_val_image_path, '*'))
num_files = len(val_image_files)
print(f"Found {num_files} images to process.")

In [ ]:
val_verification_results = {}

In [ ]:
MASK_EXTENSION = '.png'

for img_path in tqdm(val_image_files, desc="Verifying Bounding Boxes"):

    full_filename = os.path.basename(img_path)
    base_name_no_ext = os.path.splitext(full_filename)[0]
    mask_filename = base_name_no_ext + MASK_EXTENSION
    mask_path = os.path.join(local_val_mask_path, mask_filename)


    if not os.path.exists(mask_path):
        val_verification_results[full_filename] = "MASK NOT FOUND"
        continue

    # Get Bounding Boxes
    try:
        bboxes = mask_to_multiple_bboxes(mask_path, min_area_threshold=5)

        # Visualize and Save
        visualize_bbox_on_image(img_path, bboxes)
        val_verification_results[full_filename] = f"SUCCESS: {len(bboxes)} box(es)"

    except Exception as e:
        val_verification_results[full_filename] = f"ERROR: {e}"

print("\nBatch Verification Complete.")

In [ ]:
val_verification_results

In [ ]:
i = 0

for key, value in val_verification_results.items():
  if value == "SUCCESS: 0 box(es)":
    print(f"Key: {key}, Value: {value}")
    i += 1

print(i)